In [1]:
#analyze file
#grabbed torvik data from 2014-24 (no 20 / covid)
#looking for correlations in column fields that would predict the elite 8 flag

In [2]:
# import lib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

In [3]:
df_torvik = pd.read_csv('data/barttorvik_predict_L0.csv')

display(df_torvik.head(5))
display(df_torvik.info())


,Rk,Team,Conf,Seed,G,Win,AdjOE,AdjDE,Barthag,EFG%,...,FTRD,2P%,2P%D,3P%,3P%D,3PR,3PRD,Adj T.,WAB,bluePower
0,1,Houston,B12,1.0,34,30,124.8,88,0.9823,52.7,...,34.1,49,43.9,39.8,30.9,34.5,43.1,61.4,11.6,8
1,NaN,"1 seed, ✅",NaN,NaN,NaN,NaN,9,1,1,88,...,211,267,5,2,33,296,306,360,2,NaN
2,2,Duke,ACC,1.0,34,31,128.5,91.3,0.9807,57.4,...,25.4,58,43.4,37.7,30.9,45.4,37.9,65.7,9.6,8
3,NaN,"1 seed, ✅",NaN,NaN,NaN,NaN,2,4,2,5,...,21,8,3,20,33,51,142,267,5,NaN
4,3,Auburn,SEC,1.0,33,28,129,93.7,0.9756,55.7,...,39.2,56.1,47.2,36.8,29.2,40.6,34.8,67.8,12.5,8


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Rk         377 non-null    object 
 1   Team       445 non-null    object 
 2   Conf       377 non-null    object 
 3   Seed       68 non-null     float64
 4   G          377 non-null    object 
 5   Win        377 non-null    object 
 6   AdjOE      741 non-null    object 
 7   AdjDE      741 non-null    object 
 8   Barthag    741 non-null    object 
 9   EFG%       741 non-null    object 
 10  EFGD%      741 non-null    object 
 11  TOR        741 non-null    object 
 12  TORD       741 non-null    object 
 13  ORB        741 non-null    object 
 14  DRB        741 non-null    object 
 15  FTR        741 non-null    object 
 16  FTRD       741 non-null    object 
 17  2P%        741 non-null    object 
 18  2P%D       741 non-null    object 
 19  3P%        741 non-null    object 
 20  3P%D      

None

In [4]:
#remove any rows where the column "Rk" is equal to NaN

df_torvik.dropna(subset=["Rk"], inplace=True)

display(df_torvik.info())

<class 'pandas.core.frame.DataFrame'>
Index: 377 entries, 0 to 739
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Rk         377 non-null    object 
 1   Team       377 non-null    object 
 2   Conf       377 non-null    object 
 3   Seed       68 non-null     float64
 4   G          377 non-null    object 
 5   Win        377 non-null    object 
 6   AdjOE      377 non-null    object 
 7   AdjDE      377 non-null    object 
 8   Barthag    377 non-null    object 
 9   EFG%       377 non-null    object 
 10  EFGD%      377 non-null    object 
 11  TOR        377 non-null    object 
 12  TORD       377 non-null    object 
 13  ORB        377 non-null    object 
 14  DRB        377 non-null    object 
 15  FTR        377 non-null    object 
 16  FTRD       377 non-null    object 
 17  2P%        377 non-null    object 
 18  2P%D       377 non-null    object 
 19  3P%        377 non-null    object 
 20  3P%D       377 

None

In [5]:
#delete rows where the "Rk" column has the value "Rk"

df_torvik = df_torvik[df_torvik["Rk"] != "Rk"]


In [6]:
# Replace NaN Seed teams; these didn't make the tournament

df_torvik = df_torvik[df_torvik["Seed"].notna() & (df_torvik["Seed"] > 0)]

In [7]:
#drop G columns

df_torvik = df_torvik.drop(columns=["Rk","G"])

In [8]:
#columns 5 to 25 to float using astype(float

numbers = ["Seed","Win","AdjOE","AdjDE","Barthag","EFG%","EFGD%","TOR","TORD","ORB","DRB","FTR","FTRD","2P%","2P%D","3P%","3P%D","3PR","3PRD","Adj T.","bluePower"]


df_torvik[numbers] = df_torvik[numbers].apply(pd.to_numeric, errors="coerce")



In [9]:
#invert the ranks/seeds, so that higher values are transformed into lower ones, aligning better with their actual value.

#df_torvik["Rk"] = df_torvik["Rk"].max() - df_torvik["Rk"] + 1
df_torvik["Seed"] = df_torvik["Seed"].max() - df_torvik["Seed"] + 1



In [10]:
# teams that have under 12 wins get a 12

df_torvik["Win"] = df_torvik["Win"].fillna(12)  # Replace NaN with 12

In [11]:
# "Conf" column in df_torvik by replacing 'ind' with 'Ind'

df_torvik["Conf"] = df_torvik["Conf"].str.replace(r"^ind$", "Ind", case=False, regex=True)

In [12]:
# one-hot encode the "Conf" column

df_torvik = pd.get_dummies(df_torvik, columns=["Conf"], drop_first=False)



In [13]:
#new column named "AdjJOD" at index #6. The value is 1 if "AdjOE" is > 119 and if "AdjDE" is <94. Else the value is 0.

df_torvik.insert(6, "AdjOD", np.where((df_torvik["AdjOE"] > 116) & (df_torvik["AdjDE"] < 96), 1, 0))

#this just shows where there is overlap of elite 8 and adjOD
#df_adj_jod_1 = df_torvik[df_torvik["AdjOD"] == 1]
#display(df_adj_jod_1.head(25))


In [14]:
# drop WAB

df_torvik = df_torvik.drop(columns=["WAB"])
display(df_torvik.info())

<class 'pandas.core.frame.DataFrame'>
Index: 68 entries, 0 to 612
Data columns (total 54 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Team       68 non-null     object 
 1   Seed       68 non-null     float64
 2   Win        68 non-null     float64
 3   AdjOE      68 non-null     float64
 4   AdjDE      68 non-null     float64
 5   Barthag    68 non-null     float64
 6   AdjOD      68 non-null     int64  
 7   EFG%       68 non-null     float64
 8   EFGD%      68 non-null     float64
 9   TOR        68 non-null     float64
 10  TORD       68 non-null     float64
 11  ORB        68 non-null     float64
 12  DRB        68 non-null     float64
 13  FTR        68 non-null     float64
 14  FTRD       68 non-null     float64
 15  2P%        68 non-null     float64
 16  2P%D       68 non-null     float64
 17  3P%        68 non-null     float64
 18  3P%D       68 non-null     float64
 19  3PR        68 non-null     float64
 20  3PRD       68 no

None

In [15]:
#swap T/F with 1/0

#add this is there is no Cond Ind teams#####
#df_torvik["Conf_A10"] = 0
#df_torvik["Conf_ACC"] = 0
df_torvik["Conf_AE"] = 0
df_torvik["Conf_ASun"] = 0
#df_torvik["Conf_Amer"] = 0
#df_torvik["Conf_B10"] = 0
#df_torvik["Conf_B12"] = 0
#df_torvik["Conf_BE"] = 0
df_torvik["Conf_BSky"] = 0
df_torvik["Conf_BSth"] = 0
#df_torvik["Conf_BW"] = 0
df_torvik["Conf_CAA"] = 0
#df_torvik["Conf_CUSA"] = 0
df_torvik["Conf_Horz"] = 0
df_torvik["Conf_Ivy"] = 0
df_torvik["Conf_MAAC"] = 0
df_torvik["Conf_MAC"] = 0
df_torvik["Conf_MEAC"] = 0
#df_torvik["Conf_MVC"] = 0
#df_torvik["Conf_MWC"] = 0
df_torvik["Conf_NEC"] = 0
df_torvik["Conf_OVC"] = 0
df_torvik["Conf_P12"] = 0
df_torvik["Conf_Pat"] = 0
df_torvik["Conf_SB"] = 0
df_torvik["Conf_SC"] = 0
#df_torvik["Conf_SEC"] = 0
df_torvik["Conf_SWAC"] = 0
#df_torvik["Conf_Slnd"] = 0
df_torvik["Conf_Sum"] = 0
#df_torvik["Conf_WAC"] = 0
#df_torvik["Conf_WCC"] = 0
#############

# Extract column names that start with "Conf"
CONF = [col for col in df_torvik.columns if col.startswith("Conf")]

# Print or use CONF as needed
print(CONF)

df_torvik[CONF] = df_torvik[CONF].astype(int)


['Conf_A10', 'Conf_ACC', 'Conf_AE', 'Conf_ASun', 'Conf_Amer', 'Conf_B10', 'Conf_B12', 'Conf_BE', 'Conf_BSky', 'Conf_BSth', 'Conf_BW', 'Conf_CAA', 'Conf_CUSA', 'Conf_Horz', 'Conf_Ivy', 'Conf_MAAC', 'Conf_MAC', 'Conf_MEAC', 'Conf_MVC', 'Conf_MWC', 'Conf_NEC', 'Conf_OVC', 'Conf_Pat', 'Conf_SB', 'Conf_SC', 'Conf_SEC', 'Conf_SWAC', 'Conf_Slnd', 'Conf_Sum', 'Conf_WAC', 'Conf_WCC', 'Conf_P12']


In [16]:
#To apply a log transformation to columns

df_torvik[numbers] = np.log(df_torvik[numbers] + 1)

display(df_torvik.head(5))
display(df_torvik.info())

,Team,Seed,Win,AdjOE,AdjDE,Barthag,AdjOD,EFG%,EFGD%,TOR,...,Conf_Pat,Conf_SB,Conf_SC,Conf_SEC,Conf_SWAC,Conf_Slnd,Conf_Sum,Conf_WAC,Conf_WCC,Conf_P12
0,Houston,2.833213,3.433987,4.834693,4.488636,0.684258,1,3.983413,3.826465,2.747271,...,0,0,0,0,0,0,0,0,0,0
2,Duke,2.833213,3.465736,4.863681,4.525044,0.683450,1,4.067316,3.817712,2.734368,...,0,0,0,0,0,0,0,0,0,0
4,Auburn,2.833213,3.367296,4.867534,4.550714,0.680872,1,4.037774,3.850148,2.667228,...,0,0,0,1,0,0,0,0,0,0
6,Florida,2.833213,3.433987,4.857484,4.553877,0.678693,1,4.025352,3.835142,2.772589,...,0,0,0,1,0,0,0,0,0,0
8,Alabama,2.772589,3.258097,4.856707,4.578826,0.674015,0,4.048301,3.889777,2.873565,...,0,0,0,1,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
Index: 68 entries, 0 to 612
Data columns (total 55 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Team       68 non-null     object 
 1   Seed       68 non-null     float64
 2   Win        68 non-null     float64
 3   AdjOE      68 non-null     float64
 4   AdjDE      68 non-null     float64
 5   Barthag    68 non-null     float64
 6   AdjOD      68 non-null     int64  
 7   EFG%       68 non-null     float64
 8   EFGD%      68 non-null     float64
 9   TOR        68 non-null     float64
 10  TORD       68 non-null     float64
 11  ORB        68 non-null     float64
 12  DRB        68 non-null     float64
 13  FTR        68 non-null     float64
 14  FTRD       68 non-null     float64
 15  2P%        68 non-null     float64
 16  2P%D       68 non-null     float64
 17  3P%        68 non-null     float64
 18  3P%D       68 non-null     float64
 19  3PR        68 non-null     float64
 20  3PRD       68 no

None

In [17]:
df_torvik['bluePower_Barthag'] = df_torvik['bluePower'] * df_torvik['Barthag']
df_torvik['bluePower_Seed'] = df_torvik['bluePower'] * df_torvik['Seed']

# Display the updated df_torvik with the new interaction feature
display(df_torvik[['Team', 'bluePower', 'Barthag', 'bluePower_Barthag']].head())
display(df_torvik[['Team', 'bluePower', 'Seed', 'bluePower_Seed']].head())

,Team,bluePower,Barthag,bluePower_Barthag
0,Houston,2.197225,0.684258,1.503468
2,Duke,2.197225,0.683450,1.501694
4,Auburn,2.197225,0.680872,1.496029
6,Florida,2.197225,0.678693,1.491241
8,Alabama,2.197225,0.674015,1.480963


,Team,bluePower,Seed,bluePower_Seed
0,Houston,2.197225,2.833213,6.225206
2,Duke,2.197225,2.833213,6.225206
4,Auburn,2.197225,2.833213,6.225206
6,Florida,2.197225,2.833213,6.225206
8,Alabama,2.197225,2.772589,6.092000


In [18]:
# Convert to csv

df_torvik.to_csv("../L1/data/barttorvik_predict_L1.csv")